$$
M_a = \frac{1}{n} \sum_{i=1}^{n} x_i
$$

In [27]:
from binance.spot import Spot
import time
import pandas as pd
import numpy as np

client = Spot(base_url='https://testnet.binance.vision', api_key='LIMHyRg559t5sZFzOHMu9Ev8oGljAj6uoApouYwGNnsmaWC4FFAtR3cu7K2bPTvm', api_secret='YrZhktWxLmYKezclreCqufT3XTIiKCsgwzwtFDOFz6OcoZreMhxhsBxfqN0PPyaM')

df = pd.DataFrame()
count1 = 0
count2 = 0
df['op'] = 0
df['count'] = 0

def dati():
    global df
    price = client.avg_price(symbol = 'BTCUSDT')
    price_df = pd.json_normalize(price)
    df = pd.concat([df, price_df])
    df.reset_index(inplace=True, drop=True)
    df['op'] = df['op'].fillna(0)

def media_mobile():
    df["x"]=df["price"].rolling(window=5, min_periods=2).mean()
    df["y"]=df["price"].rolling(window=21, min_periods=4).mean()
    df['x'] = df['x'].fillna(0)
    df['y'] = df['y'].fillna(0)

def logica():
    global count1
    global count2
    count1 = count1 + 1
    # Se x > y e x < y a t-1 compra
    if df['x'].loc[x] > df['y'].loc[x]:
        if x == 0:
            y = x
        if x > 0:
            y = x - 1
            if df['x'].loc[y] <= df['y'].loc[y]:
                df['op'].loc[x] = 'BUY'
                df['count'].loc[x] = count1
    # Se x > y e x > y a t-1 neutro
    if df['x'].loc[x] > df['y'].loc[x]:
        if x == 0:
            y = x
        if x > 0:
            y = x - 1
            if df['x'].loc[y] > df['y'].loc[y]:
                df['op'].loc[x] = 0
    # Se x < y e x > y a t-1 vende
    if df['x'].loc[x] <= df['y'].loc[x]:  
        if x == 0:
            y = x
        if x > 0:
            y = x - 1
            if df['x'].loc[y] > df['y'].loc[y]:
                df['op'].loc[x] = 'SELL'
                df['count'].loc[x] = count1
    # Se x < y e x < y a t-1 neutro
    if df['x'].loc[x] <= df['y'].loc[x]:
        if x == 0:
            y = x
        if x > 0:
            y = x - 1
            if df['x'].loc[y] <= df['y'].loc[y]:
                df['op'].loc[x] = 0

def esecuzione():
    global count2
    count2 = count2 + 1
    if df['op'].loc[x] == 'BUY':
        if df['count'].loc[x] > count2:
            df('buy')
            
for x in range(0, 60):
    time.sleep(1)
    dati()
    media_mobile()
    logica()
    
df['w'] = 0

for x in range(len(df)):
    if df['op'].loc[x] == 'BUY':
        df['w'].loc[x] = df['price'].loc[x]

df['z'] = 0

for x in range(len(df)):
    if df['op'].loc[x] == 'SELL':
        df['z'].loc[x] = df['price'].loc[x]

df = df.astype({'w': float, 'z': float})
        
print(df['w'].sum())
print(df['z'].sum())

189589.92572322
189592.22259481
